In [1]:
pip install transformers accelerate datasets peft trl sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvji

In [2]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
file_path = "ai-medical-chatbot.csv" 
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "yousefsaeedian/ai-medical-chatbot",
    file_path,
)
df = df.sample(n=200000, random_state=42)

train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(test_df)

/tmp/ipykernel_19/3997573760.py:7: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [4]:
from transformers import BioGptTokenizer, BioGptForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
import torch

model_name = "microsoft/biogpt"
tokenizer = BioGptTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Avoid pad token issues


model = BioGptForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map='auto',
    torch_dtype=torch.float16,
)

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = [
    "c_attn",   # Attention mechanism (query, key, value combined)
    "c_proj",   # Output projection after attention
    "q_proj",   # Query projection (attention mechanism)
    "v_proj",   # Value projection (attention mechanism)
    "mlp.c_fc", # Feed-forward layer (first linear transformation)
    "mlp.c_proj", # Feed-forward layer (second linear transformation)
    "ln_1",     # Layer normalization before attention
    "ln_2",     # Layer normalization after attention
]
)

model = get_peft_model(model, peft_config)


def format_example(example):
    return {
        "text": f"### Patient: {example['Patient']}\n### Doctor: {example['Doctor']}"
    }

dataset = train_dataset.map(format_example)

checkpointing_args = {"use_reentrant": False}

training_args = SFTConfig(
    output_dir="./biogpt-qlora-medical",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    packing=True,
    report_to="none",
    max_seq_length=1024,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs=checkpointing_args
)



trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=training_args,
)

trainer.train()

trainer.model.save_pretrained("./biogpt-qlora-medical")
tokenizer.save_pretrained("./biogpt-qlora-medical")
val_dataset.save_to_disk("./val_dataset")

2025-05-11 12:28:48.696196: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746966528.919104      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746966528.986313      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

Tokenizing train dataset:   0%|          | 0/180000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/180000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,4.670800
20,4.450800
30,4.285000
40,4.225000
50,4.156100
60,4.099700
70,4.092500
80,4.038700
90,4.006200
100,3.962300


Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]